In [1]:
%load_ext autoreload
%autoreload 2

In [119]:
import ipywidgets as w
from IPython.display import clear_output, display
import matplotlib.pyplot as plt

In [3]:
import ez_torch as ez
import torch.nn as nn
import torchvision
from ez_torch.modules import Module, SpatialUVOffsetTransformer

In [4]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.feature_model = nn.Sequential(
            torchvision.models.resnet18(
                pretrained=False,
                progress=False,
                num_classes=32,
            ),
            nn.ReLU(),
        )
        # self.feature_model = nn.Sequential(
        #     nn.Flatten(),
        #     nn.Linear(28 * 28, 32),
        #     nn.ReLU(),
        # )
        self.transform = SpatialUVOffsetTransformer(
            32,
            uv_resolution_shape=(10, 10),
        )

    def criterion(self, y, y_target):
        return F.binary_cross_entropy(y, y_target)

    def forward(self, x):
        x_3d = x.repeat([1, 3, 1, 1])
        X_features = self.feature_model(x_3d)
        X_transformed = self.transform([X_features, x])
        return X_transformed

In [11]:
Model().ui

TrainableUI()

In [116]:
out = w.Output()
out

Output()

In [170]:
for i in range(100):
    z = 2

In [204]:
import bqplot as bq
from IPython.display import display

x_sc = bq.LinearScale(min=0, max=2 * np.pi)
y_sc = bq.LinearScale(min=-1, max=1)

ax_x = bq.Axis(label='Test X', scale=x_sc, tick_format='0.0f')
ax_y = bq.Axis(label='Test Y', scale=y_sc,
               orientation='vertical', tick_format='0.2f')

line = bq.Scatter(x=[0], y=[0], scales={'x': x_sc, 'y': y_sc}, colors=['red', 'yellow'])

fig = bq.Figure(axes=[ax_x, ax_y], marks=[line])

display(fig)

Figure(axes=[Axis(label='Test X', scale=LinearScale(max=6.283185307179586, min=0.0), tick_format='0.0f'), Axis…

In [205]:
# fig.layout.width = '300px'
fig.title = 'Test title'

In [207]:
import time
for i in range(100):
    x = np.linspace(0, (i + 1) / 100 * np.pi*2, 100)
    y = np.sin(x)
    line.x = x
    line.y = y
    time.sleep(0.01)